In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime as dt

In [ ]:
sns.set_style('darkgrid')
sns.set(font_scale = 1.2)

In [ ]:
data = pd.read_csv('/kaggle/input/marketing-data/marketing_data.csv')
data.head()

In [ ]:
data.info()

### Cleaning Variables

In [ ]:
data.rename({' Income ' : 'Income'}, axis = 1, inplace = True)

data['Income'] = data['Income'].str.replace('$', '')
data['Income'] = data['Income'].str.replace(',', '').astype(float)

In [ ]:
data['Dt_Customer'] = pd.to_datetime(data['Dt_Customer'])

### Null Values

In [ ]:
data.isnull().sum()

In [ ]:
data['Income'] = data['Income'].fillna(data['Income'].mean())

### Create more variables

+ At what age customer enrolled in the company

In [ ]:
data['Customer_Age_When_Enrolled'] = data['Dt_Customer'].dt.year - data['Year_Birth']

+ Customers enrolled after age 80

In [ ]:
data[data['Customer_Age_When_Enrolled'] >= 80]

In [ ]:
data['Total Spent'] = (data['MntFishProducts'] + data['MntWines'] + data['MntSweetProducts']
                       + data['MntFruits'] + data['MntMeatProducts'] + data['MntGoldProds'] )

In [ ]:
data['Total Purchases'] = (data['NumDealsPurchases'] + data['NumWebPurchases'] + data['NumStorePurchases'] +
                            data['NumCatalogPurchases'])

#### Reducing columns

+ Merging columns Kids at home and Teenagers at home

In [ ]:
data['Kids_Teen_at_home'] = data['Kidhome'] + data['Teenhome']
data = data.drop(['Kidhome', 'Teenhome'], axis = 1)

# Exploratory Data Analysis

#### Birth year

In [ ]:
sns.boxplot(data = data['Year_Birth'], color = 'r')

+ Remove outliers 

In [ ]:
data = data[data['Year_Birth'] >= 1920]

In [ ]:
sns.violinplot(data = data['Year_Birth'], color = 'purple')

### Education

In [ ]:
plt.figure(figsize = (7, 6))
sns.countplot(x = data['Education'], palette = 'rocket_r')

### Marital Status

In [ ]:
m = data['Marital_Status'].value_counts().to_frame('Count')

sns.set_palette('Blues_r')
plt.figure(figsize = (6, 6))
plt.pie(m['Count'], labels = m.index, explode = (0.1, 0, 0, 0, 0, 1, 2.5, 4), shadow = True, autopct = '%1.1f%%')
plt.show()

### Income 

In [ ]:
sns.boxplot(data = data['Income'], color = 'gold')

In [ ]:
data = data[data['Income'] < 200000]

In [ ]:
sns.distplot(data['Income'], color = 'aqua')

### Education vs Salary

In [ ]:
sns.set_palette('rocket')
plt.figure(figsize = (12, 7))
sns.violinplot(y = data['Income'], x = data['Education'])

### Income vs Gold

In [ ]:
sns.regplot(x = data['MntGoldProds'], y = data['Income'], color = 'gold')
plt.xlabel('Amount Spent on Gold')

### Income vs Total Spent in last two years

In [ ]:
sns.regplot(x = data['Total Spent'], y = data['Income'], color = 'red')

### Recency

In [ ]:
plt.figure(figsize = (7, 5))
sns.distplot(data['Recency'], color = 'm')

### Customer age when enrolled

In [ ]:
plt.figure(figsize = (7, 5))
sns.distplot(data['Customer_Age_When_Enrolled'], color = 'brown')
plt.xlabel('Customer Age When Enrolled')

### Number of Customers from each Country

In [ ]:
plt.figure(figsize = (7, 5))
sns.countplot(x = data['Country'], palette = 'icefire_r')
plt.ylabel('No. of Customers')

## Which country has more number of purchases made?

In [ ]:
plt.figure(figsize = (10, 7))
sns.violinplot(x = data['Country'], y = data['Total Purchases'], palette = 'coolwarm')

In [ ]:
pm = data[['Total Purchases', 'Country']].groupby(['Country']).agg([sum])

sns.set_palette('Spectral')
plt.figure(figsize = (7, 7))
plt.pie(pm['Total Purchases']['sum'], labels = pm.index, explode = (0, 0.5, 0, 0.3, 1, 0, 0, 0.5),
        shadow = True, autopct = '%1.1f%%')
plt.show()

### Number of Enrollments in each month and year

In [ ]:
plt.figure(figsize = (8, 5))
sns.countplot(x = data['Dt_Customer'].dt.month)
plt.xlabel('Month')
plt.ylabel('Enrollments')

In [ ]:
sns.countplot(x = data['Dt_Customer'].dt.year)
plt.xlabel('Year')
plt.ylabel('Enrollments')

## Purchases made through each source

In [ ]:
plt.figure(figsize = (8, 6))
tp = data[['NumDealsPurchases', 'NumWebPurchases', 'NumStorePurchases', 'NumCatalogPurchases']].agg([sum])

sns.barplot(x = tp.T.index, y = tp.T['sum'], palette = 'mako_r')
plt.gca().set_xticklabels(['Deals', 'Web', 'Store', 'Catalog'])
plt.xlabel('Purchase Through')
plt.ylabel('Purchases')
plt.show()

## Income vs Purchases

In [ ]:
plt.figure(figsize = (7, 5))
sns.regplot(x = data['Total Purchases'], y = data['Income'], color = 'grey')

## Income vs Wine

In [ ]:
plt.figure(figsize = (7, 5))
sns.regplot(x = data['MntWines'], y = data['Income'], color = 'hotpink')
plt.xlabel('Spent on Wine')

## Proportion of customers that spent almost non on gold

In [ ]:
bins = [0, 10, 30, 50, 100, 200, 500]
b = data[['ID']].copy()
b['Gold Bins'] = pd.cut(data['MntGoldProds'], bins = bins)
b = b.groupby('Gold Bins').agg([len])

sns.set_palette('viridis_r')
plt.figure(figsize = (10, 7))
plt.pie(b['ID']['len'], labels = b.index,  explode = (0.1, 0, 0.2, 0, 0, 0.3), shadow = True, autopct = '%1.1f%%')
plt.show()

## Amount spent on different Products

In [ ]:
prod = data[['MntFruits', 'MntSweetProducts', 'MntFishProducts', 'MntMeatProducts', 'MntGoldProds']].agg([sum]).T

sns.barplot(x = prod.index, y = prod['sum'])
plt.gca().set_xticklabels(['Fruits', 'Sweet', 'Fish', 'Meat', 'Gold'])
plt.xlabel('Products')
plt.ylabel('Amount')

# Work still in Progess.....


# Upvote if you like my work.....